# pQTL v PD two sample MR

In [1]:
library("TwoSampleMR")
library("ieugwasr")

TwoSampleMR 0.5.0 - Major update!
[>] List of changes and information about backwards compatibility here:
    https://mrcieu.github.io/TwoSampleMR/articles/gwas2020.html
[>] Browse available data at https://gwas.mrcieu.ac.uk


API: public: http://gwas-api.mrcieu.ac.uk/


Attaching package: ‘ieugwasr’


The following object is masked from ‘package:TwoSampleMR’:

    ld_matrix




# Traits of interest

## Exposure GWAS

### Get the available GWASs in the database

In [2]:
ao = gwasinfo()
str(ao)

Classes ‘GwasInfo’, ‘tbl_df’, ‘tbl’ and 'data.frame':	11016 obs. of  20 variables:
 $ id         : chr  "prot-a-1363" "ukb-a-567" "ubm-a-1567" "prot-a-3030" ...
 $ trait      : chr  "Heterogeneous nuclear ribonucleoprotein A/B" "Diagnoses - main ICD10: M24 Other specific joint derangements" "NET100 0411" "Tumor necrosis factor alpha-induced protein 3" ...
 $ sample_size: int  3301 337199 7916 3301 3301 3301 3301 3301 7916 148695 ...
 $ nsnp       : int  10534735 10894596 11734354 10534735 10534735 10534735 10534735 10534735 11734354 9851867 ...
 $ year       : int  2018 2017 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ consortium : chr  "NA" "Neale Lab" "NA" "NA" ...
 $ author     : chr  "Sun BB" "Neale" "Elliott LT" "Sun BB" ...
 $ note       : chr  "NA" "NA" "NA" "NA" ...
 $ access     : chr  "public" "public" "public" "public" ...
 $ mr         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ sex        : chr  "Males and Females" "Males and Females" "Males" "Males and Females" ...
 $ pmid       

In [3]:
head(ao)

id,trait,sample_size,nsnp,year,consortium,author,note,access,mr,sex,pmid,priority,population,unit,category,subcategory,ncase,ncontrol,sd
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>
prot-a-1363,Heterogeneous nuclear ribonucleoprotein A/B,3301,10534735,2018,NA,Sun BB,NA,public,1,Males and Females,29875488,0,European,NA,Immune system,Protein,NA,NA,NA
ukb-a-567,Diagnoses - main ICD10: M24 Other specific joint derangements,337199,10894596,2017,Neale Lab,Neale,NA,public,1,Males and Females,NA,1,European,SD,NA,NA,838,336361,NA
ubm-a-1567,NET100 0411,7916,11734354,2018,NA,Elliott LT,NA,public,1,Males,30305740,0,European,SD,Continuous,Psychiatric / neurological,NA,NA,NA
prot-a-3030,Tumor necrosis factor alpha-induced protein 3,3301,10534735,2018,NA,Sun BB,NA,public,1,Males and Females,29875488,0,European,NA,Immune system,Protein,NA,NA,NA
prot-a-2084,Ribosyldihydronicotinamide dehydrogenase [quinone],3301,10534735,2018,NA,Sun BB,NA,public,1,Males and Females,29875488,0,European,NA,Immune system,Protein,NA,NA,NA
prot-a-1970,Mitochondrial ubiquitin ligase activator of NFKB 1,3301,10534735,2018,NA,Sun BB,NA,public,1,Males and Females,29875488,0,European,NA,Immune system,Protein,NA,NA,NA


### Extract the list of pQTLs to lookup

In [4]:
exp_traits = unique(ao[grepl("prot-a-", ao$id),]$id)
str(exp_traits)

 chr [1:3282] "prot-a-1363" "prot-a-3030" "prot-a-2084" "prot-a-1970" ...


### Example - get descriptive statistics of pQTL

In [5]:
gwasinfo("prot-a-1363")

id,trait,sample_size,nsnp,year,consortium,author,note,pmid,mr,unit,sex,population,access,subcategory,category,priority
<chr>,<chr>,<int>,<int>,<int>,<lgl>,<chr>,<lgl>,<int>,<int>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
prot-a-1363,Heterogeneous nuclear ribonucleoprotein A/B,3301,10534735,2018,NA,Sun BB,NA,29875488,1,NA,Males and Females,European,public,Protein,Immune system,0


## Outcome GWAS

In [6]:
outcome_traits = c("ukb-b-6548", "ukb-b-956", "ukb-b-16943")
### Double check IDs
for(id in outcome_traits) {
    cat("id: ", id, "\n")
    print(gwasinfo(id))
    cat("\n\n")
}

id:  ukb-b-6548 
          id                                    trait sample_size    nsnp year
1 ukb-b-6548 Illnesses of mother: Parkinson's disease      422464 9851867 2018
  consortium       author
1    MRC-IEU Ben Elsworth
                                                                                note
1 20110#11: Output from GWAS pipeline using Phesant derived variables from UKBiobank
  mr unit ncontrol               sex population access subcategory category
1  1   SD   415466 Males and Females   European public          NA   Binary
  priority ncase
1        1  6998


id:  ukb-b-956 
         id                                    trait sample_size    nsnp year
1 ukb-b-956 Illnesses of father: Parkinson's disease      399089 9851867 2018
  consortium       author
1    MRC-IEU Ben Elsworth
                                                                                note
1 20107#11: Output from GWAS pipeline using Phesant derived variables from UKBiobank
  mr unit ncontrol   

# Extract the instruments from the pQTL GWAS

In [7]:
exp.df = tophits(
  id=exp_traits,
  pval=5e-08, # default
  clump=1, # flag to clump or not
  r2=0.001, # default
  kb=10000 # default
)
exp.df

n,chr,se,beta,p,position,id,rsid,ea,nea,eaf,trait
<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3301,12,0.0775,0.5223,1.58489e-11,6493911,prot-a-2927,rs2302255,T,C,0.02556,Tapasin-related protein
3301,12,0.1097,0.7648,3.16228e-12,6842463,prot-a-2927,rs187826664,A,G,0.01544,Tapasin-related protein
3301,14,0.0835,0.5086,1.14815e-09,94844947,prot-a-391,rs28929474,T,C,0.02269,C-C motif chemokine 15
3301,17,0.0329,0.1848,1.99526e-08,34273986,prot-a-391,rs4796139,G,A,0.16963,C-C motif chemokine 15
3301,15,0.0248,-0.1751,1.69824e-12,49871878,prot-a-1097,rs12899680,G,C,0.48101,Fibroblast growth factor 7
3301,15,0.0258,-0.1856,6.76083e-13,50364841,prot-a-1097,rs7162172,C,T,0.3919,Fibroblast growth factor 7
3301,5,0.0734,-0.4155,1.51356e-08,40972211,prot-a-1582,rs74480769,G,A,0.0317,Integrin alpha-5
3301,2,0.0276,0.1862,1.51356e-11,234633242,prot-a-1008,rs202203863,G,C,0.28143,Tissue Factor
3301,12,0.0270,0.2262,5.01187e-17,6809896,prot-a-2267,rs11064321,C,G,0.39732,PILR alpha-associated neural protein


# Find the instruments in the outcome GWASs

In [8]:
snp_list = unique(exp.df$rsid)
str(snp_list)

 chr [1:2341] "rs2302255" "rs187826664" "rs28929474" "rs4796139" ...


In [10]:
out.df = associations(
  variants=snp_list, #rsids to lookup
  id=outcome_traits, #traits to lookup
  proxies=1, #Look for proxies in outcome.
  r2=0.8, #proxy r2 required
  align_alleles=1, #default
  palindromes=1, #default
  maf_threshold=0.3, #default
)
out.df

beta,p,se,position,n,chr,id,rsid,ea,nea,eaf,target_snp,proxy_snp,proxy,target_a1,target_a2,proxy_a1,proxy_a2,trait
<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
-9.13499e-05,0.8100000,0.000370449,34273986,422464,17,ukb-b-6548,rs4796139,G,A,0.169253,rs4796139,rs4796139,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
1.39679e-04,0.6200000,0.000279838,49871878,422464,15,ukb-b-6548,rs12899680,G,C,0.488347,rs12899680,rs12899680,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
1.61423e-05,0.9600000,0.000286311,50364841,422464,15,ukb-b-6548,rs7162172,C,T,0.392394,rs7162172,rs7162172,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-4.66942e-04,0.1300000,0.000308050,234647317,422464,2,ukb-b-6548,rs202203863,A,G,0.284917,rs202203863,rs6722076,TRUE,G,C,A,G,Illnesses of mother: Parkinson's disease
-3.55963e-04,0.3000000,0.000341617,54324995,422464,19,ukb-b-6548,rs4632248,T,G,0.211187,rs4632248,rs4632248,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-3.29484e-04,0.3300000,0.000339585,54327869,422464,19,ukb-b-6548,rs10418046,G,T,0.212714,rs10418046,rs10418046,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-6.57964e-04,0.2600000,0.000585913,120475302,422464,9,ukb-b-6548,rs4986790,G,A,0.059608,rs4986790,rs4986790,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-3.66131e-04,0.2900000,0.000346931,186395113,422464,3,ukb-b-6548,rs2228243,G,A,0.200199,rs2228243,rs2228243,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-1.63538e-04,0.7099990,0.000443282,7690617,422464,12,ukb-b-6548,rs4883275,C,T,0.112781,rs4883275,rs4883275,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease


beta,p,se,position,n,chr,id,rsid,ea,nea,eaf,target_snp,proxy_snp,proxy,target_a1,target_a2,proxy_a1,proxy_a2,trait
<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
-9.13499e-05,0.8100000,0.000370449,34273986,422464,17,ukb-b-6548,rs4796139,G,A,0.169253,rs4796139,rs4796139,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
1.39679e-04,0.6200000,0.000279838,49871878,422464,15,ukb-b-6548,rs12899680,G,C,0.488347,rs12899680,rs12899680,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
1.61423e-05,0.9600000,0.000286311,50364841,422464,15,ukb-b-6548,rs7162172,C,T,0.392394,rs7162172,rs7162172,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-4.66942e-04,0.1300000,0.000308050,234647317,422464,2,ukb-b-6548,rs202203863,A,G,0.284917,rs202203863,rs6722076,TRUE,G,C,A,G,Illnesses of mother: Parkinson's disease
-3.55963e-04,0.3000000,0.000341617,54324995,422464,19,ukb-b-6548,rs4632248,T,G,0.211187,rs4632248,rs4632248,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-3.29484e-04,0.3300000,0.000339585,54327869,422464,19,ukb-b-6548,rs10418046,G,T,0.212714,rs10418046,rs10418046,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-6.57964e-04,0.2600000,0.000585913,120475302,422464,9,ukb-b-6548,rs4986790,G,A,0.059608,rs4986790,rs4986790,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-3.66131e-04,0.2900000,0.000346931,186395113,422464,3,ukb-b-6548,rs2228243,G,A,0.200199,rs2228243,rs2228243,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
-1.63538e-04,0.7099990,0.000443282,7690617,422464,12,ukb-b-6548,rs4883275,C,T,0.112781,rs4883275,rs4883275,FALSE,NA,NA,NA,NA,Illnesses of mother: Parkinson's disease
